# Run CNN Model<br>
Steps:<br>
1. Read dataset file
2. Create training & test datasets
3. Create & run CNN model
4. Evaluation
5. Save output into file

# Step 1. Read Dataset File

## Import libraries

In [182]:
# Import file and data libraries
import math
import pickle
import numpy as np
import pandas as pd
from collections import Counter

# Import Matthews Correlation Coefficient module (MCC)
from sklearn.metrics import matthews_corrcoef

# Import libraries
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.convolutional import AveragePooling1D
from keras.layers.merge import concatenate
from sklearn.preprocessing import MinMaxScaler

## Read dataset file

In [183]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [184]:
# Read X & Y scaled datasets from Pickle files
dataset_path = "/content/drive/My Drive/Colab Notebooks/3. Code/3. Compare/Dataset/"

with open(dataset_path + "Dataset V2.pickle", "rb") as f:
    X = pickle.load(f)  # Contains Date, HL_Avg, Volume, HL_Avg_Norm, Volume_Norm
    Y = pickle.load(f)  # Contains Y_Avg, X_Last, Y_Diff, Y_Band, Y_Gap, Y_Days, Y_Avg_Norm, X_Last_Norm, Y_Band_Norm
    Metadata = pickle.load(f) # Contains the Company ID, ticker, sector and Dataset ID
f.close()

# Step 2. Create training & test datasets

## Make X & Y uniform length

In [185]:
# The min length of X datasets is 1230: min([len(X[i]) for i in range(len(X))]) 
# Make X has uniform length by taking the last 1230 values: (take the normalised price and volume)
# X[0][-1230:] means taking the last 1230 values
X_Uniform_Length = [X[i][-1230:][["HL_Avg_Norm","Volume_Norm"]] for i in range(len(X))]

# Convert to Numpy array 
X_CNN = np.array([np.array(X_Uniform_Length[i]) for i in range(len(X_Uniform_Length))])
X_CNN.shape

(3989, 1230, 2)

In [186]:
# This is CNN, so the output (Y) is categorical. Use the Y Band, not the price.
Y_CNN_Band = np.array([Y[i][3] for i in range(len(Y))]) # Y[i][3] is the Y Band
print(Y_CNN_Band.shape, Y_CNN_Band[0])

(3989,) U1


## Convert Y into numeric

In [187]:
# CNN requires Y to be an array of 10 numbers (1 and 0s)
# Convert Y from bands (U1, U2, D1, ...) to a class array like this: [1,0,0,0,0,0,0,0,0,0] 
def Convert_Band_To_Class(Band):
    Class = [0,0,0,0,0,0,0,0,0,0]
    if   Band == "D5": Class[0] = 1
    elif Band == "D4": Class[1] = 1
    elif Band == "D3": Class[2] = 1
    elif Band == "D2": Class[3] = 1
    elif Band == "D1": Class[4] = 1
    elif Band == "U1": Class[5] = 1
    elif Band == "U2": Class[6] = 1
    elif Band == "U3": Class[7] = 1
    elif Band == "U4": Class[8] = 1
    elif Band == "U5": Class[9] = 1
    return Class

In [188]:
# Convert a class array like [1,0,0,0,0,0,0,0,0,0] to price band like U1, U2, D1, D2
def Convert_Class_To_Band(Class):
    Band = ["D5","D4","D3","D2","D1","U1","U2","U3","U4","U5"]
    return Band[Class]

In [189]:
Y_CNN = np.array([Convert_Band_To_Class(Y_CNN_Band[i]) for i in range(len(Y_CNN_Band))])
Y_CNN

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

## Split into training and test

In [190]:
# Split into Training and Test dataasets
# Test dataset: the first 1000, Training dataset: the remaining (3000 - 11 = 2989)
X_train = X_CNN[1000:]
X_test  = X_CNN[:1000]
Y_train = Y_CNN[1000:]
Y_test  = Y_CNN[:1000]
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(2989, 1230, 2) (1000, 1230, 2) (2989, 10) (1000, 10)


# Step 3. Create & run CNN Model

## Create CNN model

In [191]:
n_steps = X_CNN.shape[1]
n_features = X_CNN.shape[2]
print(n_steps, n_features)

1230 2


In [192]:
# File name
File_Name = "CNN 3L 8 32 128 AP4 K402010 D1024 256 DO.csv"

In [193]:
# Create 1D CNN model
model = Sequential()
model.add(Conv1D(filters=8, kernel_size=40, activation='relu', input_shape=(n_steps, n_features)))
model.add(AveragePooling1D(pool_size=4))

model.add(Conv1D(filters=32, kernel_size=20, activation='relu', input_shape=(n_steps, n_features)))
model.add(AveragePooling1D(pool_size=4))

model.add(Conv1D(filters=128, kernel_size=10, activation='relu', input_shape=(n_steps, n_features)))
model.add(AveragePooling1D(pool_size=4))

model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(rate=0.2))
model.add(Dense(256, activation="relu"))
model.add(Dropout(rate=0.2))
model.add(Dense(10, activation="softmax"))
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [194]:
# Fit model
model.fit(X_train, Y_train, epochs=10)

Epoch 1/10
94/94 [==============================] - 3s 28ms/step - loss: 2.2304
Epoch 2/10
94/94 [==============================] - 3s 29ms/step - loss: 2.2084
Epoch 3/10
94/94 [==============================] - 3s 29ms/step - loss: 2.1951
Epoch 4/10
94/94 [==============================] - 3s 30ms/step - loss: 2.1891
Epoch 5/10
94/94 [==============================] - 3s 30ms/step - loss: 2.1758
Epoch 6/10
94/94 [==============================] - 3s 29ms/step - loss: 2.1645
Epoch 7/10
94/94 [==============================] - 3s 30ms/step - loss: 2.1511
Epoch 8/10
94/94 [==============================] - 3s 29ms/step - loss: 2.1440
Epoch 9/10
94/94 [==============================] - 3s 30ms/step - loss: 2.1236
Epoch 10/10
94/94 [==============================] - 3s 29ms/step - loss: 2.0965


## Forecast the price band

In [195]:
# Forecast
F = model.predict(X_test)

In [196]:
F_Band_List = []
Y_Band_List = []
Status_List = []
for i in range(len(F)):
    F_Band = Convert_Class_To_Band(np.argmax(F[i]))
    Y_Band = Convert_Class_To_Band(np.argmax(Y_test[i]))
    Status = int(F_Band == Y_Band)
    #print(F_Band, Y_Band, Status)

    F_Band_List.append(F_Band)
    Y_Band_List.append(Y_Band)
    Status_List.append(Status)

# Step 4. Evaluation

In [197]:
# Get the count of each band
c = Counter(F_Band_List)
print(c.keys(), c.values())

dict_keys(['U2', 'U5', 'U3', 'U1', 'D3', 'D5', 'U4', 'D2', 'D1']) dict_values([78, 259, 12, 623, 4, 8, 4, 7, 5])


In [198]:
## Calculate %Correct
print(sum(Status_List))

180


In [199]:
# Calculate MCC
matthews_corrcoef(Y_Band_List, F_Band_List) # 0.025504582511996728

0.057592710311828506

# Step 5. Save output into CSV file

In [200]:
Output_File = ""
for i in range(len(Y_Band_List)):
    output = Y_Band_List[i] + "," +  F_Band_List[i] + "," + str(Status_List[i]) + "\n"
    Output_File = Output_File + output   

In [201]:
print(File_Name)

CNN 3L 8 32 128 AP4 K402010 D1024 256 DO.csv


In [202]:
# Save to CSV file
File_Path = "/content/drive/My Drive/Colab Notebooks/3. Code/3. Compare/Output/5. CNN/"
f = open(File_Path + File_Name, "w")
f.write(Output_File)
f.close()